In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive')


Mounted at /content/drive


In [ ]:
import argparse
import time
import os
import numpy as np
import random
import torch
import RUL.MutiSource.model_mfsan as model_mfsan
import RUL.MutiSource.data_loader as data_loader
from RUL.MutiSource.utils import get_free_gpu, weight_init, weight_init2, log_in_file, visualize_total_loss, save_model,score_cal,rmse_cal,load_model
import math
from torch.autograd import Variable
import torch.nn.functional as F
import RUL.MutiSource.utils as utils
import pandas as pd
pd.set_option('mode.chained_assignment', None)

In [ ]:
import torchvision.datasets as datasets
from torch.utils.data import SubsetRandomSampler, DataLoader
from torchvision import transforms
import torch
import logging
import pandas as pd
import numpy as np
from sklearn import preprocessing
import torch
from torch.utils.data import Dataset

def gen_sequence(id_df, seq_length, seq_cols):
    data_matrix = id_df[seq_cols].values
    num_elements = data_matrix.shape[0] #矩阵行数

    for start, stop in zip(range(0, num_elements - seq_length), range(seq_length, num_elements)):
        yield data_matrix[start:stop, :]


def gen_labels(id_df, seq_length, label):
    data_matrix = id_df[label].values
    num_elements = data_matrix.shape[0]

    return data_matrix[seq_length:num_elements, :]



In [ ]:
#path
current_dir = "/content/drive/MyDrive/RUL"
train_FD001_path = current_dir +'/cmapss/train_FD001.csv'
test_FD001_path = current_dir +'/cmapss/test_FD001.csv'
RUL_FD001_path = current_dir+'/cmapss/RUL_FD001.txt'
FD001_path = [train_FD001_path, test_FD001_path, RUL_FD001_path]

train_FD002_path = current_dir +'/cmapss/train_FD002.csv'
test_FD002_path = current_dir +'/cmapss/test_FD002.csv'
RUL_FD002_path = current_dir +'/cmapss/RUL_FD002.txt'
FD002_path = [train_FD002_path, test_FD002_path, RUL_FD002_path]

train_FD003_path = current_dir +'/cmapss/train_FD003.csv'
test_FD003_path = current_dir +'/cmapss/test_FD003.csv'
RUL_FD003_path = current_dir +'/cmapss/RUL_FD003.txt'
FD003_path = [train_FD003_path, test_FD003_path, RUL_FD003_path]

train_FD004_path =current_dir +'/cmapss/train_FD004.csv'
test_FD004_path = current_dir +'/cmapss/test_FD004.csv'
RUL_FD004_path = current_dir +'/cmapss/RUL_FD004.txt'
FD004_path = [train_FD004_path, test_FD004_path, RUL_FD004_path]

## Assign columns name
cols = ['unit_nr', 'cycles', 'os_1', 'os_2', 'os_3']
cols += ['sensor_{0:02d}'.format(s + 1) for s in range(26)]
col_rul = ['RUL_truth']

## Read csv file to pandas dataframe
FD_path = ["none", FD001_path, FD002_path, FD003_path, FD004_path]
FD_name = ["none", "FD001", "FD002", "FD003", "FD004"]

# Training settings
sequence_length = 30
batch_size = 512
epochs = 10
lr = [0.001, 0.01,0.05]
momentum = 0.9
cuda = True
seed = 8
log_interval = 5
l2_decay = 5e-4

sensor_drop = ['sensor_01', 'sensor_05', 'sensor_06', 'sensor_10', 'sensor_16', 'sensor_18', 'sensor_19']

source_path = ["none", FD_path[1], FD_path[2], FD_path[3], FD_path[4]]  #此处选择source 数据集 1 2 3 分别对应FD001-FD004
target_path = ["none", FD_path[1], FD_path[2], FD_path[3], FD_path[4]]
datasetset_name = ["none", "FD001", "FD002", "FD003", "FD004"]

torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)



In [ ]:
source_chosen = ['None',1,3,4]
target_chosen = ['None',2]


source1_name = datasetset_name[source_chosen[1]]
source2_name = datasetset_name[source_chosen[2]]
source3_name = datasetset_name[source_chosen[3]]

target_train_loader = data_loader.load_training(target_path[target_chosen[1]], sequence_length, sensor_drop,
                                                batch_size)


In [ ]:

for inputs,label in target_train_loader:
    print(inputs,label)

In [ ]:
piecewise_lin_ref = 125
data_path_list = FD002_path


train_FD = pd.read_csv(data_path_list[0], sep=' ', header=None,
                        names=cols, index_col=False)
test_FD = pd.read_csv(data_path_list[1], sep=' ', header=None,
                      names=cols, index_col=False)
RUL_FD = pd.read_csv(data_path_list[2], sep=' ', header=None,
                      names=col_rul, index_col=False)
mapper = {}
for unit_nr in train_FD['unit_nr'].unique():
    mapper[unit_nr] = train_FD['cycles'].loc[train_FD['unit_nr'] == unit_nr].max()

# calculate RUL = time.max() - time_now for each unit
train_FD['RUL'] = train_FD['unit_nr'].apply(lambda nr: mapper[nr]) - train_FD['cycles']
# print(train_FD['RUL'].describe())
# train_FD['RUL'].head(200)


# piecewise linear for RUL labels
train_FD['RUL'].loc[(train_FD['RUL'] > piecewise_lin_ref)] = piecewise_lin_ref
# print(train_FD['RUL'].describe())

mapper2 = {}
for unit_nr in train_FD['unit_nr'].unique():
    mapper2[unit_nr] = train_FD['RUL'].loc[train_FD['unit_nr'] == unit_nr].max()
# calculate RUL = time.max() - time_now for each unit
train_FD['RUL'] = (train_FD['RUL'])/train_FD['unit_nr'].apply(lambda nr: mapper2[nr])
# piecewise linear for RUL labels
# train_FD['RUL'].describe()
#print(train_FD['RUL'])

In [ ]:

# Cut max RUL ground truth
RUL_FD['RUL_truth'].loc[(RUL_FD['RUL_truth'] > piecewise_lin_ref)] = piecewise_lin_ref
RUL_FD['RUL_truth']

0       18
1       79
2      106
3      110
4       15
      ... 
254    122
255    125
256     56
257    125
258     51
Name: RUL_truth, Length: 259, dtype: int64

In [ ]:
## Excluse columns which only have NaN as value
# nan_cols = ['sensor_{0:02d}'.format(s + 22) for s in range(5)]
cols_nan = train_FD.columns[train_FD.isna().any()].tolist()
# print('Columns with all nan: \n' + str(cols_nan) + '\n')
cols_const = [col for col in train_FD.columns if len(train_FD[col].unique()) <= 2]  ###os3列在这个时候已经被去掉了
# print('Columns with all const values*******: \n' + str(cols_const) + '\n')

## Drop exclusive columns
train_FD = train_FD.drop(columns=cols_const + cols_nan + sensor_drop)
test_FD = test_FD.drop(columns=cols_const + cols_nan + sensor_drop)

# 对数据进行Min—Max变换

## preprocessing(normailization for the neural networks)
min_max_scaler = preprocessing.MinMaxScaler()
# for the training set
# train_FD['cycles_norm'] = train_FD['cycles']
cols_normalize = train_FD.columns.difference(['unit_nr', 'cycles', 'os_1', 'os_2', 'RUL']) #求差集保留train_FD.columns

norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_FD[cols_normalize]),
                              columns=cols_normalize,
                              index=train_FD.index)
join_df = train_FD[train_FD.columns.difference(cols_normalize)].join(norm_train_df)
train_FD = join_df.reindex(columns=train_FD.columns)

# for the test set
# test_FD['cycles_norm'] = test_FD['cycles']
cols_normalize_test = test_FD.columns.difference(['unit_nr', 'cycles', 'os_1', 'os_2'])
# print ("cols_normalize_test", cols_normalize_test)
norm_test_df = pd.DataFrame(min_max_scaler.transform(test_FD[cols_normalize_test]),
                            columns=cols_normalize_test,
                            index=test_FD.index)
test_join_df = test_FD[test_FD.columns.difference(cols_normalize_test)].join(norm_test_df)
test_FD = test_join_df.reindex(columns=test_FD.columns)
test_FD = test_FD.reset_index(drop=True)



# Specify the columns to be used
sequence_cols_train = train_FD.columns.difference(['unit_nr', 'cycles', 'os_1', 'os_2', 'RUL'])
sequence_cols_test = test_FD.columns.difference(['unit_nr', 'os_1', 'os_2', 'cycles'])
# print(train_FD.columns.tolist(),test_FD.columns.tolist(),sequence_cols_train,sequence_cols_test)
## generator for the sequences
# transform each id of the train dataset in a sequence
seq_gen = (list(gen_sequence(train_FD[train_FD['unit_nr'] == id], sequence_length, sequence_cols_train))
            for id in train_FD['unit_nr'].unique())

# generate sequences and convert to numpy array in training set
seq_array_train = np.concatenate(list(seq_gen)).astype(np.float32)
seq_array_train = seq_array_train.transpose(0, 2, 1)  # shape = (samples, sensors, sequences)
# print("seq_array_train.shape", self.seq_array_train.shape)
# generate label of training samples
label_gen = [gen_labels(train_FD[train_FD['unit_nr'] == id], sequence_length, ['RUL'])
              for id in train_FD['unit_nr'].unique()]
label_array_train = np.concatenate(label_gen).astype(np.float32)


In [ ]:
# generate sequences and convert to numpy array in test set (only the last sequence for each engine in test set)
seq_array_test_last = [test_FD[test_FD['unit_nr'] == id][sequence_cols_test].values[-sequence_length:]
                        for id in test_FD['unit_nr'].unique() if
                        len(test_FD[test_FD['unit_nr'] == id]) >= sequence_length]
seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
seq_array_test_last = seq_array_test_last.transpose(0, 2, 1)  

In [ ]:
y_mask = [len(test_FD[test_FD['unit_nr'] == id]) >= sequence_length for id in test_FD['unit_nr'].unique()]
print(len(y_mask))

259


In [ ]:
mapper1 = {}
for unit_nr in test_FD['unit_nr'].unique():
    mapper1[unit_nr] = test_FD['cycles'].loc[test_FD['unit_nr'] == unit_nr].max()
mapper1_df = pd.DataFrame(list(mapper1.items()))


In [ ]:
testmax = test_FD['unit_nr'].drop_duplicates(keep='first')

# testmax['2'] = mapper1_df[1]
# testmax = mapper1_df[1]
testmax = testmax.reset_index()
testmax.loc[:, 'maxlen'] = 0
testmax['maxlen'] = mapper1_df[1]
# for i in range(len(mapper1_df[1])):
#   testmax['maxlen'][i] = mapper1_df[1][i]
# pd.merge(testmax, mapper1_df,left_index=True, right_index=True, how='outer')
# pd.concat([testmax, mapper1_df], axis=1，join=outer )
testmax

,index,unit_nr,maxlen
0,0,1,258
1,258,2,55
2,313,3,165
3,478,4,86
4,564,5,148
...,...,...,...
254,33362,255,150
255,33512,256,59
256,33571,257,199
257,33770,258,98


In [ ]:
label_array_test_last = RUL_FD['RUL_truth'][y_mask].values/(RUL_FD['RUL_truth'][y_mask].values+testmax['maxlen'][y_mask].values)
label_array_test = label_array_test_last.reshape(label_array_test_last.shape[0], 1).astype(np.float32)
#print("label_array_test.shape", self.label_array_test.shape)
#print(self.label_array_test.shape)

In [ ]:
def train(model):
    global mmd_loss1,mmd_loss2,mmd_loss3
    print("--------------------------MFSAN --------------------------------")

    f_mfsan_train = log_in_file("/mfsan_train_log.log")

    source1_loader = data_loader.load_training(source_path[source_chosen[1]], sequence_length, sensor_drop,
                                                batch_size)
    source2_loader = data_loader.load_training(source_path[source_chosen[2]], sequence_length, sensor_drop,
                                                batch_size)
    source3_loader = data_loader.load_training(source_path[source_chosen[3]], sequence_length, sensor_drop,
                                               batch_size)

    source1_len = len(source1_loader)
    source2_len = len(source2_loader)
    source3_len = len(source3_loader)

    target_len = len(target_train_loader)
    max_len = max(source1_len,source2_len,source3_len,target_len)
    # print("111111",source1_len,source2_len,target_len,max_len)

    batch_src1,batch_src2,batch_src3,batch_tar = 0,0,0,0
    running_loss_scr1, running_loss_scr2 ,running_loss_scr3= 0, 0, 0
    list_src1, list_src2, list_src3, list_tar = list(enumerate(source1_loader)), list(enumerate(source2_loader)), list(enumerate(source3_loader)),list(enumerate(target_train_loader))

    optimizer = torch.optim.SGD([
        {'params': model.sharedNet.parameters(), 'lr': lr[1]},

        {'params': model.rul_fc_son1.parameters(), 'lr': lr[1]},
        {'params': model.rul_fc_son2.parameters(), 'lr': lr[2]},

        {'params': model.sonnet1.parameters(), 'lr': lr[1]},
        {'params': model.sonnet2.parameters(), 'lr': lr[2]},

    ], lr=lr[2], momentum=momentum, weight_decay=l2_decay)



    for batch in range(1, max_len):
        model.train()

        p = (batch - 1) / (max_len)
        alpha = 2. / (1. + np.exp(-10 * p)) - 1
        optimizer = utils.optimizer_scheduler(optimizer=optimizer, p=p)
        optimizer.zero_grad()

        #####scr1 tgt
        _, (source_data1, source_label1) = list_src1[batch_src1]
        _, (target_data1, _) = list_tar[batch_tar]


        if cuda:
            source_data1, source_label1 = source_data1.cuda(), source_label1.type(torch.FloatTensor).cuda()
            target_data1 = target_data1.cuda()
        source_data1, source_label1 = Variable(source_data1), Variable(source_label1) #tensor不能反向传播，variable可以反向传播
        target_data1 = Variable(target_data1)

        rul_loss1, mmd_loss1, l1_loss1 = model(source_data1, target_data1, source_label1, alpha, mark=1)
        gamma = 2 / (1 + math.exp(-10 * (batch) / (max_len))) - 1
        loss1 = rul_loss1 + gamma * (mmd_loss1 + l1_loss1)
        loss1.backward()
        optimizer.step()
        running_loss_scr1 += loss1.item()

        batch_src1 += 1
        if batch_src1 >= len(list_src1)-1:
            batch_src1 = 0

        if batch_tar >= len(list_tar)-1:
            batch_tar = 0

        if batch % log_interval == 0:
            print('Train source1 iter: {} [({:.0f}%)]\tLoss: {:.6f}\tRUL_Loss: {:.6f}\tmmd_Loss: {:.6f}\tl1_Loss: {:.6f}'.format(
                batch, 100. * batch/ max_len, loss1.item(), rul_loss1.item(), mmd_loss1.item(), l1_loss1.item()), file=f_mfsan_train, flush=True)

        # print("batch_src1,batch_tar",batch_src1,batch_tar)

        #####scr2 tgt
        _, (source_data2, source_label2) = list_src2[batch_src2]
        _, (target_data2, _) = list_tar[batch_tar]

        if cuda:
            source_data2, source_label2 = source_data2.cuda(), source_label2.type(torch.FloatTensor).cuda()
            target_data2 = target_data2.cuda()
        source_data2, source_label2 = Variable(source_data2), Variable(source_label2)
        target_data2 = Variable(target_data2)
        optimizer.zero_grad()

        rul_loss2, mmd_loss2, l1_loss2 = model(source_data2, target_data2, source_label2, alpha, mark=2)
        gamma = 2 / (1 + math.exp(-10 * (batch) / (max_len))) - 1
        loss2 = rul_loss2 + gamma * (mmd_loss2 + l1_loss2)
        loss2.backward()
        optimizer.step()
        running_loss_scr2 += loss2.item()

        batch_src2 += 1
        if batch_src2 >= len(list_src2)-1:
            batch_src2 = 0


        if batch_tar >= len(list_tar)-1:
            batch_tar = 0
        # print("batch_src2,batch_tar", batch_src2, batch_tar)

        if batch % log_interval == 0:
            print(
                'Train source2 iter: {} [({:.0f}%)]\tLoss: {:.6f}\tRUL_Loss: {:.6f}\tmmd_Loss: {:.6f}\tl1_Loss: {:.6f}'.format(
                    batch, 100. * batch / max_len, loss2.item(), rul_loss2.item(), mmd_loss2.item(), l1_loss2.item()), file=f_mfsan_train, flush=True)

        #####scr3 tgt
        _, (source_data3, source_label3) = list_src3[batch_src3]
        _, (target_data3, _) = list_tar[batch_tar]

        if cuda:
            source_data3, source_label3 = source_data3.cuda(), source_label3.type(torch.FloatTensor).cuda()
            target_data3 = target_data3.cuda()
        source_data3, source_label3 = Variable(source_data3), Variable(source_label3)
        target_data3 = Variable(target_data3)
        optimizer.zero_grad()

        rul_loss3, mmd_loss3, l1_loss3 = model(source_data3, target_data3, source_label3, alpha, mark=3)
        gamma = 2 / (1 + math.exp(-10 * (batch) / (max_len))) - 1
        loss3 = rul_loss3 + gamma * (mmd_loss3 + l1_loss3)
        loss3.backward()
        optimizer.step()
        running_loss_scr3 += loss3.item()

        batch_src3 += 1
        if batch_src3 >= len(list_src3) - 1:
            batch_src3 = 0

        batch_tar += 1
        if batch_tar >= len(list_tar) - 1:
            batch_tar = 0
        # print("batch_src2,batch_tar", batch_src2, batch_tar)

        if batch % log_interval == 0:
            print(
                'Train source3 iter: {} [({:.0f}%)]\tLoss: {:.6f}\tRUL_Loss: {:.6f}\tmmd_Loss: {:.6f}\tl1_Loss: {:.6f}\n\n'.format(
                    batch, 100. * batch / max_len, loss3.item(), rul_loss3.item(), mmd_loss3.item(), l1_loss3.item()),
                file=f_mfsan_train, flush=True)

    f_mfsan_train.close()
    epoch_loss_scr1 = running_loss_scr1 / (max_len - 1)
    epoch_loss_scr2 = running_loss_scr2 / (max_len - 1)
    epoch_loss_scr3 = running_loss_scr3 / (max_len - 1)

    return epoch_loss_scr1, epoch_loss_scr2, epoch_loss_scr3


def test(model):
    model.eval()

    f_mfsan_test = log_in_file("/mfsan_test_log.log")

    with torch.no_grad():
        for data, target in target_test_loader:
            if cuda:
                data, target = data.cuda(), target.type(torch.FloatTensor).cuda()
            data, target = Variable(data), Variable(target)
            pred1, pred2, pred3 = model(data)

            pred1, pred2, pred3 =pred1.detach().cpu().numpy().squeeze(1), pred2.detach().cpu().numpy().squeeze(1), pred3.detach().cpu().numpy().squeeze(1)
            target = target.detach().cpu().numpy()

            rmse1 = rmse_cal(pred1, target)  # sum up batch loss
            score1 = score_cal(pred1, target)
            rmse2 = rmse_cal(pred2, target) # sum up batch loss
            score2 = score_cal(pred2, target)
            rmse3 = rmse_cal(pred3, target)  # sum up batch loss
            score3 = score_cal(pred3, target)
            #w1 = (mmd_loss1-0.5) ** (-2)
            #w2 = (mmd_loss2-0.5) ** (-2)
            #w3 = (mmd_loss3-0.5) ** (-2)
            w1 = mmd_loss1 ** (-1)
            w2 = mmd_loss2 ** (-1)
            w3 = mmd_loss3 ** (-1)
            #w1 = mmd_loss1
            #w2 = mmd_loss2
            #w3 = mmd_loss3
            w1 = w1.cpu().numpy()
            w2 = w2.cpu().numpy()
            w3 = w3.cpu().numpy()
            ws=w1+w2+w3

            #pred = (w1*pred1 + w2*pred2 + w3*pred3) / ws
            pred=(pred1+pred2+pred3)/3
            rmse = rmse_cal(pred, target) # sum up batch loss
            score = score_cal(pred, target)


        print(target_name, '\nTest set: Mutil_source rmse: {:.4f}, Mutil_source score: {:.4f}'.format(rmse, score), file=f_mfsan_test, flush=True)
        print('source1 rmse {:.4f}, source2 rmse {:.4f}, source3 rmse {:.4f}'.format(rmse1, rmse2,rmse3), file=f_mfsan_test, flush=True)
        print('source1 score {:.4f}, source2 score {:.4f}, source3 score {:.4f}\n\n'.format(score1, score2,score3), file=f_mfsan_test, flush=True)
    return rmse, score,rmse1,score1,rmse2,score2,rmse3,score3,pred1, pred2, pred3, pred,target


In [ ]:
model = model_mfsan.MFSAN().apply(weight_init)

In [ ]:
model

MFSAN(
  (sharedNet): CFE(
    (conv1): Conv1d(14, 32, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv1d(32, 64, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
    (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv1d(64, 128, kernel_size=(2,), stride=(1,), bias=False)
    (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (sonnet1): DFE(
    (rnn): LSTM(128, 100, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (sonnet2): DFE(
    (rnn): LSTM(128, 100, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (sonnet3): DFE(
    (rnn): LSTM(128, 100, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (rul_fc_son1): Linear(in_features